# In this notebook, we use Supervised Machine Learning with classification to identify Fraudulent Medicare providers using data from CMS that has been preprocessed using Data Wrangler

## Setup

Import required libraries (install imblearn using pip if not present)

In [ ]:
!pip install imblearn

In [ ]:
import numpy as np 
import pandas as pd
import boto3
import os
import sagemaker
import seaborn as sns
import matplotlib.pyplot as plt
import io
import sklearn
from math import sqrt
from sagemaker import get_execution_role
from sagemaker import RandomCutForest
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.amazon.amazon_estimator import get_image_uri
from sklearn.datasets import dump_svmlight_file  
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.datasets import dump_svmlight_file   
from collections import Counter

Enable the ability to see all columns and rows of data if the data size is big

In [ ]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'fraud-detect-demo'
role = get_execution_role()
s3_client = boto3.client("s3")

Let's start by reading in the entire preprocessed medicare data set prepared for classification

In [ ]:
!gzip -d processed_data_classification.csv.gz

In [ ]:
data = pd.read_csv('processed_data_classification.csv', delimiter=',')
data.head()

## Investigate and process the data

Check data for any nulls

In [ ]:
data.isnull().values.any()

Check for imbalance

In [ ]:
data['fraudulent_provider'].value_counts()

We see that the majority of data is non-fraudulent. We will need to rebalance the data using sampling techniques that are designed specifically for imbalanced problems to improve the performance of the model.We use the Random Under Sampler and Over Sampling techniques from imblearn to do this (http://glemaitre.github.io/imbalanced-learn/api.html)

First, remove column headers from data as SageMaker does not need headers for processing csv files

In [ ]:
feature_columns = data.columns[1:]
label_column = data.columns[0]

features = data[feature_columns].values.astype('float32')
labels = (data[label_column].values).astype('float32')

We will split our dataset into a train and test to evaluate the performance of our models. Since the data is highly imbalanced, it is important to stratify across the data sets to ensure an even distribution.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.1, stratify=labels)

## Apply SMOTE

The ratio in oversampling and the sampling strategy for undersampling are very important in improving the performance of the models. We have selected ratios based ased on research from https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0225-0 for this dataset. However, try to expirement with different ratios to see the impact

In [ ]:
over = SMOTE(sampling_strategy=0.25)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X_smote, y_smote = pipeline.fit_resample(X_train, y_train)

In [ ]:
print(sorted(Counter(y_smote).items()))

In [ ]:
X_smote_train, X_smote_validation, y_smote_train, y_smote_validation = train_test_split(
    X_smote, y_smote, test_size=0.1, stratify=y_smote)

## Training and Prediction - Supervised learning (classification)

We use a supervised learning algorithm for classifcation using Amazon XGBoost 

### Prepare Data and Upload to S3

We first copy the data to an in-memory buffer and then upload the data to S3 in libsvm format (XGBoost can take either libsvm or csv files as input)

In [ ]:
buf = io.BytesIO()

sklearn.datasets.dump_svmlight_file(X_smote_train, y_smote_train, buf)
buf.seek(0);

Now we upload the data to S3 using boto3.

In [ ]:
key = 'fraud-dataset'
subdir = 'base'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', subdir, key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}/{}'.format(bucket, prefix, subdir, key)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

In [ ]:
buf = io.BytesIO()
sklearn.datasets.dump_svmlight_file(X_smote_validation, y_smote_validation, buf)
buf.seek(0);

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', subdir, key)).upload_fileobj(buf)

s3_validation_data = 's3://{}/{}/validation/{}/{}'.format(bucket, prefix, subdir, key)
print('Uploaded validation data location: {}'.format(s3_validation_data))

We can now train using SageMaker's built-in XGBoost algorithm. To specify the XGBoost algorithm, we use a utility function to obtain its URI. 

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")

SageMaker abstracts training via Estimators. We can pass the classifier and parameters along with hyperparameters to the estimator, and fit the estimator to the data in S3. An important parameter here is `scale_pos_weight` which scales the weights of the positive vs. negative class examples. This is crucial to do in an imbalanced dataset like the one we are using here, otherwise the majority class would dominate the learning.

The other hyperparameters seen here were based on the results of the Hyperparameter Optimization performed using SageMaker. We describe that technique in the next section of this notebook

In [ ]:
# Because the data set is so highly skewed, we set the scale position weight conservatively,
# as sqrt(num_nonfraud/num_fraud).
# Other recommendations for the scale_pos_weight are setting it to (num_nonfraud/num_fraud).
scale_pos_weight = sqrt(np.count_nonzero(y_train==0)/np.count_nonzero(y_train))
hyperparams = {
    "max_depth":7,
    "subsample":0.8,
    "num_round":145,
    "eta":0.82,
    "gamma":4,
    "min_child_weight":41.08,
    "silent":0,
    "objective":'binary:logistic',
    "eval_metric":'auc',
    "scale_pos_weight": scale_pos_weight
}

clf = sagemaker.estimator.Estimator(container,
                                    get_execution_role(),
                                    hyperparameters=hyperparams,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=session)

clf.fit({'train': s3_train_data, 'validation': s3_validation_data})

### Host Classifier

Now we deploy the estimator to an endpoint.

In [ ]:
csv_serializer = CSVSerializer()
xgb_predictor = clf.deploy(initial_instance_count=1,
                       endpoint_name="fraud-detection-endpoint-xgb",
                       instance_type='ml.m4.xlarge', 
                       serializer=csv_serializer)

For reruns after deployment use: endpoint_name="fraud-detection-endpoint-xgb" predictor=sagemaker.predictor.Predictor(endpoint_name, sagemaker_session=session)

## Evaluation

Once we have trained the model we can use it to make predictions for the test set.

In [ ]:
# Because we have a large test set, we call predict on smaller batches
def predict(current_predictor, df, rows=500):
    split_array = np.array_split(df, int(df.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, current_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [ ]:
raw_preds = predict(xgb_predictor, X_test)

We will use a few measures from the scikit-learn package to evaluate the performance of our model. When dealing with an imbalanced dataset, we need to choose metrics that take into account the frequency of each class in the data.

We will use [balanced accuracy score](https://scikit-learn.org/stable/modules/model_evaluation.html#balanced-accuracy-score)


we can bring a balance between the metrics again by adjusting our classification threshold (threshold between labeling a point as fraud or not). We can try different thresholds to see if they affect the result of the classification. 

In [ ]:
# Calculate balanced accuracy scores for different threshold values
for thres in np.linspace(0.1, 0.99, num=10):
    smote_thres_preds = np.where(raw_preds > thres, 1, 0)
    print("Threshold: {:.1f}".format(thres))
    print("Balanced accuracy = {:.3f}".format(balanced_accuracy_score(y_test, smote_thres_preds)))

In [ ]:
# use the best thresholds from the above
y_preds = np.where(raw_preds > 0.99, 1, 0)
print("Balanced accuracy = {}".format(balanced_accuracy_score(y_test, y_preds)))

Apart from single-value metrics, it's also useful to look at metrics that indicate performance per class. A confusion matrix, and per-class precision, recall and f1-score can also provide more information about the model's performance.

In [ ]:
def plot_confusion_matrix(y_true, y_predicted):

    cm  = confusion_matrix(y_true, y_predicted)
    # Get the per-class normalized value for each cell
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # We color each cell according to its normalized value, annotate with exact counts.
    ax = sns.heatmap(cm_norm, annot=cm, fmt="d")
    ax.set(xticklabels=["non-fraud", "fraud"], yticklabels=["non-fraud", "fraud"])
    ax.set_ylim([0,2])
    plt.title('Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
    plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_preds)

In [ ]:
print(classification_report(
    y_test, y_preds, target_names=['non-fraud', 'fraud']))

In [ ]:
# # Ouput test result for creating a QuickSight dashboard
# yy_preds = np.expand_dims(y_preds, 1)
# yy_test = np.expand_dims(y_test, 1)
# test_data_result = np.concatenate((yy_preds, yy_test, X_test), axis=1)

# test_headers = list(data.columns.values)
# test_headers.insert(0, "prediction_result")

# for_quicksight = pd.DataFrame(test_data_result).to_csv("for_quicksight.csv", header=test_headers, index=False)

## Hyperparameter Optimization (hpo)

"Hyperparameters" can dramtically affect the performance of the trained models.  Picking the best hyperparameter settings  is very time consuming. We will use SageMaker hyperparameter tuning to automate the  process of finding hyperparameters that can optimize model performance. Specifically, we specify a range, or a list of possible values in the case of categorical hyperparameters, for each of the hyperparameter that we plan to tune. SageMaker hyperparameter tuning will automatically launch multiple training jobs with different hyperparameter settings, evaluate results of those training jobs based on a predefined "objective metric", and select the hyperparameter settings for future attempts based on previous results. For each hyperparameter tuning job, we will give it a budget (max number of training jobs) and it will complete once that many training jobs have been executed.

In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


hyperparameter_ranges = {
    "min_child_weight": ContinuousParameter(1, 120),
    "max_depth": IntegerParameter(1, 10),
    "num_round":IntegerParameter(1, 500)
}

objective_metric_name = "validation:auc"

tuner = HyperparameterTuner(
    clf, objective_metric_name, hyperparameter_ranges, max_jobs=20, max_parallel_jobs=3
)

s3_input_train = s3_train_data
s3_input_validation = s3_validation_data

tuner.fit({"train": s3_input_train, "validation": s3_input_validation}, include_cls_metadata=False)


In [ ]:
smclient = boto3.client(service_name='sagemaker')
smclient.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)['HyperParameterTuningJobStatus']

When the hpo job is completed, you can see the best traiing job and update the model with the best hyperparameters available in the SM console and generate predictions

In [ ]:
tuner.best_training_job()

In [ ]:
predictor = tuner.deploy(initial_instance_count=1,
                     instance_type='ml.m4.xlarge', 
                       serializer=csv_serializer)

In [ ]:
raw_preds = predict(predictor, X_test)

In [ ]:
for thres in np.linspace(0.5, 0.99, num=20):
    smote_thres_preds = np.where(raw_preds > thres, 1, 0)
    print("Threshold: {:.1f}".format(thres))
    print("Balanced accuracy = {:.3f}".format(balanced_accuracy_score(y_test, smote_thres_preds)))

In [ ]:
# scikit-learn expects 0/1 predictions, so we threshold our raw predictions
y_preds = np.where(raw_preds > 0.95, 1, 0)
print("Balanced accuracy = {}".format(balanced_accuracy_score(y_test, y_preds)))

In [ ]:
print(classification_report(
    y_test, y_preds, target_names=['non-fraud', 'fraud']))

If the balanceed accuracy is improved, you can update the model with the best hyperparameters as required

## Check the data for Bias

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
# converting the facet value that we wil check for bias ('female') from float to int for setting up Sagemaker clarify bias processing
train['female'] = train['female'].astype(int)

In [ ]:
from sagemaker.s3 import S3Uploader

train.to_csv("train.csv", index=False, header=False)

train_uri = S3Uploader.upload("train.csv", "s3://{}/{}".format(bucket, prefix))

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.r5.24xlarge", sagemaker_session=session
)

In [ ]:
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_uri,
    s3_output_path=bias_report_output_path,
    label="fraudulent_provider",
    headers=train.columns.to_list(),
    dataset_type="text/csv"
)

Update the `model_name` below with the `model_name` of the sagemaker endpoint you deployed. this should be available in the parameters returned from the 
`describe_endpoint_config` command in the sagemaker client smclient. To get the endpoint config you need to run the `describe_endpoint` command on the endpoint you deployed

In [ ]:
smclient = boto3.client(service_name='sagemaker')

In [ ]:
smclient.describe_endpoint(EndpointName = "fraud-detection-endpoint-xgb")

In [ ]:
smclient.describe_endpoint_config(EndpointConfigName= 'fraud-detection-endpoint-xgb')

IMPORTANT: Use the `ModelName` from the above in the `model_name` below

In [ ]:
model_config = clarify.ModelConfig(
    model_name="UpdateModelName",
    instance_type="ml.r5.24xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.75)

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="female", facet_values_or_threshold=[1]
)

In [ ]:
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

In [ ]:
bias_report_output_path

In [ ]:
boto3.resource('s3').Bucket(bucket).download_file("fraud-detect-demo/clarify-bias/report.pdf", "bias_report.pdf")

To view the bias metrics, open up the bias_report.pdf - alternatively you can view results in Studio under the expirements tab

## Evaluate which features contribute to the model predictions (Explainability)

In [ ]:
test_features = test.drop(["fraudulent_provider"], axis=1)

In [ ]:
shap_config = clarify.SHAPConfig(
    baseline=[test_features.iloc[0].values.tolist()],
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=train_uri,
    s3_output_path=explainability_output_path,
    label="fraudulent_provider",
    headers=train.columns.to_list(),
    dataset_type="text/csv",
)

In [ ]:
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

In [ ]:
explainability_output_path

In [ ]:
boto3.resource('s3').Bucket(bucket).download_file("fraud-detect-demo/clarify-explainability/report.pdf", "exp_report.pdf")

To view the analysis on explainability, open up the exp_report.pdf. Alternately, you can view bias and explainibility reports in Studio under the experiments tab

## Clean up

In [ ]:
# Uncomment to clean up endpoints
# xgb_predictor.delete_endpoint()
# predictor.delete_endpoint()


## Data Acknowledgements

The dataset used to demonstrated the fraud detection solution has been collected and analysed from CMS 

https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners/medicare-physician-other-practitioners-by-provider-and-service

